# Problem 10.1: Lossy Compression With a Variational Autoencoder (VAE)

**Course:** Data Compression With And Without Deep probabilistic models (Prof. Robert Bamler at University of Tuebingen)

- This notebook is part of Problem Set 10, published on 8 June 2022 and discussed on 15 June 2022.
- You can download the full problem set (and solutions) from the [course website](https://robamler.github.io/teaching/compress22/).

This problem guides you through the implementation of a lossy compression scheme for hand-written digits (MNIST).
We'll use a Variational Autoencoder (VAE) and the quantization method introduced in [Ballé et al., "End-to-end optimized image compression" (ICLR 2017)](https://openreview.net/forum?id=rJxdQ3jeg).
The VAE is similar to the one you implemented on the last problem set, but you can solve both problem sets independently.

## Setup

Same as on the last problem set:
let's load some libraries, set some global configurations, and load the training and test sets.

In [ ]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
USE_CUDA = False # change to True if you have a compatible GPU
RANDOM_SEED = 20220625
BATCH_SIZE = 100
EPOCHS = 50
BATCHES_BETWEEN_LOGGING = 100
LATENT_DIM = 8

In [ ]:
device = torch.device("cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

## Visualize some of the training data

Same as on the last problem set:
the training data consists of black-and-white handwritten digits with a resolution of 28x28 pixels.

In [ ]:
for batch, _ in train_loader:
    ax = plt.imshow(make_grid(batch[:64]).permute((1, 2, 0)))
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    break

## Encoder and Decoder Model

Here's where things start to differ from the last problem set.

The classes `Encoder` and `Decoder` define again two neural networks.
However, this time, the variational distribution (whose parameters are the output of the encoder network) is a uniform distribution with a fixed width.
Thus:

- `Encoder` is a neural network with weights $\phi$.
  It takes a batch of images $x$ as input and outputs a vector $\mu_\phi(x)$ that lives in the latent representation space.
  This vector parameterizes the variational distribution $Q_\phi(Z\,|\,X\!=\!x)$, which is a *uniform* (continuous) distribution over a hypercube of width 1 that is centered around $\mu_\phi(x)$.
  This means that the probability density $Q_\phi(Z\!=\!z\,|\,X\!=\!x)$ is constant (one) for all points $z$ that satisfy $|z_i-\mu_\phi(x)\leq \frac12|$ for all coordinates $i$, and the density is zero everywhere else.
- `Decoder` is the same as on the last problem set.
  It is a neural network with weights $\theta$ that takes a latent representation $z$ as input and outputs the parameters $\hat x_\theta(z)$ of the likelihood $P_\theta(X\,|\,Z\!=\!z)$.
  Here, we model the likelihood $P_\theta(X\,|\,Z\!=\!z)$ as a Bernoulli distribution, which is a fancy way of saying that each pixel $X_i$ can be either black or white, and it is white with probability $\hat x_\theta(z) \in(0,1)$.

### Your Task

Nothing to implement for you yet.
But read the above explanation and the model definitions below and make sure you understand what's going on.

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=5)
        self.fc = nn.Linear(in_features=8*8*8, out_features=LATENT_DIM)

    def forward(self, x):
        h = torch.relu(self.conv1(x))
        h = torch.relu(self.conv2(h)).view(-1, 8 * 8 * 8)
        q_mean = self.fc(h)  # (no activation function on last layer)
        return q_mean

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(in_features=LATENT_DIM, out_features=8*8*8)
        self.deconv1 = nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=5)
        self.deconv2 = nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=5, stride=2, output_padding=1)

    def forward(self, z):
        h = torch.relu(self.fc(z)).view(-1, 8, 8, 8)
        h = torch.relu(self.deconv1(h))
        return torch.sigmoid(self.deconv2(h))

## Reparameterization Gradients

We'll again use reparameterization gradients to maximize the ELBO (see Problem 8.2 (b) on Problem Set 8).
However, there are two differences this time

- The variational distribution is no longer a Gaussian distribution but instead a uniform distribution width width 1 in each dimension.
  Thus, you'll have to add uniformly distributed noise from the interval $[-\frac12, \frac12]$ (*hint:* `torch.rand(mean.shape)` draws random numbers from the interval $[0,1]$).
- The uniform noise is only introduced at training time (when `training == True` below).
  When we use the model for compression, we instead round each coordinate to the nearest integer (it would be better if we do the same at training time, but the nondifferentiable rounding operation would make gradient based optimization difficult; the uniform variational distribution with width 1 was chosen to resemble rounding on average).

### Your Task

Fill in the gap labeled `TODO` below.

In [ ]:
def reparameterize(mean, training):
    if training:
        TODO
    else:
        return torch.round(mean)

## Learnable Prior

The VAE on the last problem set had a fixed prior distribution.
This was a reasonable choice back then because we used a variational distribution with a variable width (variance).
Making the width of the prior also variable in such a situation would have lead to spurious degrees of freedom: stretching both the prior and all variational parameters by some constant $\alpha$ along some coordinate direction would have leave the ELBO invariant.
This could have lead to numerical underflows or overflows.

By contrast, on this problem set, the variational distribution has a fixed width.
This makes it crucial that the prior has a learned width.
Otherwise, the variational inference procedure would fail to yield good results (try it out if you're curious!).
More precisely, when we calculate the ELBO over the entire training set, it contains a term $D_\text{KL}\big(Q_\phi(Z) \,\big|\big|\, P(Z) \big)$ where $Q_\phi(Z):=\mathbb E_{x\sim\text{training data}} [Q_\phi(Z\,|\,X\!=\!x)]$ is called the marginal variational distribution.

We use here a Gaussian prior with zero mean and diagonal variance,
$$
  P_\theta(Z) = \mathcal N(0,\text{diag}(\gamma_1^2, \gamma_2^2,\ldots)).
$$
To ensure that the variance is positive, we parametrize it as $\gamma^2 = \exp(\texttt{prior\_log\_variance})$.
The initialization of `prior_log_variance` with $1$ is somewhat arbitrary but seems to work reasonably well here.

In [ ]:
prior_log_variance = torch.nn.Parameter(torch.ones((LATENT_DIM,))) 

## Analytic Calculation of KL-Divergence

We'll again use the following formulation of the ELBO, as given in Eq. 4 of Problem Set 8:

$$
\text{ELBO}(\phi) = \mathbb E_{Q_\phi(Z|\mathbf{X}=\mathbf{x})}\big[ \log P_\theta(\mathbf{X}\! =\!\mathbf{x}\,|\,Z)\big] - D_\text{KL}\big(Q_\phi(Z\,|\,\mathbf{X}\! =\!\mathbf{x}) \,\big|\;\!\!\big|\,  P(Z)\big).
$$

Like on the last problem set, *estimate* the first term on the right-hand side by drawing a random sample from $z\sim Q_\phi(Z\,|\,\mathbf{X}\! =\!\mathbf{x})$ and then evaluating $ \log P(\mathbf{X}\! =\!\mathbf{x}\,|\,Z\!=\!z)$ for this sample.
By contrast, the second term on the right-hand side can be evaluated analytically.

Like on the last problem set, both the prior $P(Z)$ and the variational distribution $Q_\phi(Z\,|\,\mathbf{X}\! =\!\mathbf{x})$ factorize over all dimensions~$Z_i$ of the latent representation.
Therefore, the KL-Divergence is again a sum over all dimensions:

$$
  D_\text{KL}\big(Q_\phi(Z\,|\,\mathbf{X}\! =\!\mathbf{x}) \,\big|\;\!\!\big|\,  P(Z)\big)
  = \sum_i D_\text{KL}\big(Q_\phi(Z_i\,|\,\mathbf{X}\! =\!\mathbf{x}) \,\big|\;\!\!\big|\,  P(Z_i)\big).
$$

But we now have different distributions than on the last problem set:

- $Q_\phi(Z_i\,|\,\mathbf{X}\! =\!\mathbf{x})$ is a uniform distribution of width one centered at $\mu_\phi(x)_i$.
  Thus, its probability density $Q_\phi(Z_i\!=\! z_i \,|\,\mathbf{X}\! =\!\mathbf{x}) = 1$ if $z_i \in [\mu_\phi(x)_i-\frac12, \mu_\phi(x)_i+\frac12]$ and zero otherwise.
- $P(Z_i)$ is a normal distribution with zero mean and variance $\gamma_i^2$.
  Thus, $P(Z_i\!=\!z_i) = \frac{1}{\sqrt{2\pi\gamma_i^2}} e^{- z_i^2 / (2\gamma_i^2)}$.

### Your Task

Write out the KL-diveregence $D_\text{KL}\big(Q_\phi(Z_i\,|\,\mathbf{X}\! =\!\mathbf{x}) \,\big|\;\!\!\big|\,  P(Z_i)\big)$ for a single coordinate $Z_i$ as an explicit integral over $z_i$ and solve the integral analytically (you should only have to integrate over a simple polynomial).
Then implement the result where it says `TODO` below.

In [ ]:
def kl_divergence_from_prior(q_mean):
    prior_precision = (-prior_log_variance).exp()
    return 0.5 * torch.sum(
        TODO
    )

## Training and Test procedure

The following cells define the training and testing procedure.
Notice that the loss function is defined as

$$
    \texttt{loss} = \texttt{distortion} + \beta \times \texttt{bitrate}
$$

where $\beta>0$ is a (fixed) scalar parameter that controls the trade-off between reconstruction errors ("distortion") and bit rate.
For $\beta=1$, we have $\texttt{loss}=-\text{ELBO}$.
For $\beta>1$, the model is trained to favor short bit rates even if achieving this requires higher distortion.
For $\beta<1$, the model is trained to favor low distortion (i.e., high reconstruction quality) even if achieving this requires higher bit rates.

In [ ]:
def encoder_decoder_roundtrip(encoder, decoder, x, training):
    q_mean = encoder(x)
    bitrate = kl_divergence_from_prior(q_mean)
    z = reparameterize(q_mean, training)
    x_hat = decoder(z)
    log_likelihood = -F.binary_cross_entropy(x_hat, x, reduction='sum')
    distortion = -log_likelihood
    return bitrate, distortion, x_hat

In [ ]:
encoder = Encoder().to(device)
decoder = Decoder().to(device)
if torch.cuda.device_count() > 1:
    encoder = nn.DataParallel(encoder)
    decoder = nn.DataParallel(decoder)
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=1e-3)

In [ ]:
def train(epoch, beta):
    # Set encoder and decoder models into training mode.
    encoder.train()
    decoder.train()

    total_loss = 0
    for batch_idx, (batch, _) in enumerate(train_loader):
        batch = batch.to(device)
        optimizer.zero_grad()
        bitrate, distortion, _ = encoder_decoder_roundtrip(encoder, decoder, batch, True)
        loss = distortion + beta * bitrate
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % BATCHES_BETWEEN_LOGGING == 0:
            print(
                f'Training epoch {epoch} [{batch_idx * len(batch)}/{len(train_loader.dataset)}' +
                f' ({100. * batch_idx / len(train_loader):.0f}%)]:' +
                f' loss = {loss.item() / len(batch):.6f};'
                f' bits per pixel = {bitrate.item() / (len(batch) * np.log(2) * 28**2):.6f};'
                f' distortion = {distortion.item() / len(batch):.6f}'
            )
            sys.stdout.flush()

    return total_loss / len(train_loader.dataset)

In [ ]:
def test(render_reconstructions=False):
    # Set encoder and decoder models into evaluation mode.
    encoder.eval()
    decoder.eval()

    rendered_x_hat = None
    total_bitrate = 0
    total_distortion = 0
    with torch.no_grad():
        for i, (batch, _) in enumerate(test_loader):
            batch = batch.to(device)
            bitrate, distortion, reconstruction = encoder_decoder_roundtrip(encoder, decoder, batch, False)
            total_bitrate += bitrate.item()
            total_distortion += distortion.item()
            if i == 0 and render_reconstructions:
                n = min(batch.size(0), 8)
                comparison = torch.cat([batch[:n],
                                        reconstruction.view(BATCH_SIZE, 1, 28, 28)[:n]])
                rendered_x_hat = make_grid(comparison.cpu()).permute((1, 2, 0))

    return total_bitrate / len(test_loader.dataset), total_distortion / len(test_loader.dataset), rendered_x_hat

In [ ]:
def create_figure():
    figure, (samples_plot, reconstructions_plot) = plt.subplots(1, 2)
    samples_plot.set_title('samples')
    samples_plot.xaxis.set_visible(False)
    samples_plot.yaxis.set_visible(False)
    reconstructions_plot.set_title('test images and\ntheir reconstructions')
    reconstructions_plot.xaxis.set_visible(False)
    reconstructions_plot.yaxis.set_visible(False)
    return figure, samples_plot, reconstructions_plot

## Your Task

Train the model by executing the following cell.
Try out various values for `BETA` and observe how the bit rate and the visual quality of the reconstructions change when you change `BETA`.

In [ ]:
BETA = 1.0 # TODO: change this to different values and observe the rate-distortion trade-off

torch.manual_seed(RANDOM_SEED)

def reset_parameters(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

encoder.apply(reset_parameters)
decoder.apply(reset_parameters)
prior_log_variance = torch.nn.Parameter(torch.ones((LATENT_DIM,))) 

encoder = Encoder().to(device)
decoder = Decoder().to(device)
prior_log_variance = prior_log_variance.to(device)
if torch.cuda.device_count() > 1:
    encoder = nn.DataParallel(encoder)
    decoder = nn.DataParallel(decoder)

# Note that `prior_log_variance` is now also part of the parameters over
# which we optimize:
learnable_parameters = list(encoder.parameters()) + list(decoder.parameters()) + [prior_log_variance]
optimizer = optim.Adam(learnable_parameters, lr=1e-3)

for epoch in range(1, EPOCHS + 1):
    start_time = time.time()
    show_plots = epoch == 1 or epoch % 5 == 0  

    train(epoch, BETA)
    test_bitrate, test_distortion, x_hat = test(render_reconstructions=show_plots)
    end_time = time.time()
    print(
        f'Epoch {epoch} of {EPOCHS} ({end_time - start_time:.0f} sec): ' +
        f'test bits per pixel = {test_bitrate/ (np.log(2) * 28**2):.4f}; ' +
        f'test distortion = {test_distortion:.4f}'
    )
    print(f'Prior log variances: {[i for i in prior_log_variance.cpu().detach().numpy()]}')

    if show_plots:
        figure, samples_plot, reconstructions_plot = create_figure()
        with torch.no_grad():
            sample = (torch.exp(0.5 * prior_log_variance) * torch.randn(64, LATENT_DIM)).to(device)
            sample = decoder(sample).cpu()
            samples_plot.imshow(make_grid(sample.view(64, 1, 28, 28)).permute((1, 2, 0)))
        reconstructions_plot.imshow(x_hat)
        plt.show()

## Outlook: Entropy Coding

So far, we've only implemented a model that *could* be used for lossy compression, but we haven't actually produced any compressed bit strings.
Producing the compressed bit strings isn't hard (assuming you use a library for the actual entropy coding task, such as our [`constriction` library](https://bamler-lab.github.io/constriction/)).
However, dealing with bit strings still brings along a few technicalities that would distract from the fundamentals that this problem is trying to teach.

We'll defer the actual task of encoding and decoding to and from a bit string to the next problem set.